# Laboratorio 1.1 — COVID (JHU)

Parte 1–2 resueltas sobre el daily 2022-04-18, y partes avanzadas (2–4) y series (3).


In [ ]:
import pandas as pd, numpy as np
from dateutil import parser
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import requests, io, os, math
from scipy import stats
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import statsmodels.api as sm
import pmdarima as pm
plt.rcParams['figure.figsize'] = (10,5)

## 1.1 Leer dataset (2022-04-18) y verificar
- Mostrar primeras 10
- Info general
- Valores faltantes
- Agregados por país y por país/provincia
- Provincias de China ordenado por confirmados
- Top-10 por confirmados
- Top-10 por fallecidos + país con máximo y mínimo
- Extraer aleatoriamente 50 filas, eliminar columnas 0,1,5,6,11 y guardar/leer Excel

In [ ]:
DATE_STR = '04-18-2022'
URL = f"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{DATE_STR}.csv"
raw = pd.read_csv(URL)
rename_map = {
    'Province/State':'Province_State','Country/Region':'Country_Region',
    'Last Update':'Last_Update','Latitude':'Lat','Longitude':'Long_'
}
df = raw.rename(columns=rename_map).copy()
df.head(10)

In [ ]:
df.info()
df.isna().sum()

In [ ]:
# b) por país
by_country = df.groupby('Country_Region', dropna=False).agg(
    Confirmed=('Confirmed','sum'),Deaths=('Deaths','sum'),
    Recovered=('Recovered','sum'),Active=('Active','sum')
).reset_index()
by_country.sort_values('Confirmed', ascending=False).head(10)

In [ ]:
# c) por país y provincia
by_cp = df.groupby(['Country_Region','Province_State'], dropna=False).agg(
    Confirmed=('Confirmed','sum'),Deaths=('Deaths','sum'),Recovered=('Recovered','sum')
).reset_index()
by_cp.head(10)

In [ ]:
# d) Provincias de China
china = by_cp[by_cp['Country_Region']=='China'].sort_values('Confirmed', ascending=False)
china.head(10)

In [ ]:
# e) Top-10 países por confirmados
top_confirmed = by_country.nlargest(10, 'Confirmed')
top_confirmed

In [ ]:
# f) Top-10 por fallecidos y extremos
top_deaths = by_country.nlargest(10, 'Deaths')
max_deaths = by_country.loc[by_country['Deaths'].idxmax()]
min_deaths = by_country.loc[by_country['Deaths'].idxmin()]
top_deaths, max_deaths[['Country_Region','Deaths']], min_deaths[['Country_Region','Deaths']]

In [ ]:
# g) Muestreo y drop de columnas 0,1,5,6,11
sample50 = df.sample(50, random_state=42).copy()
cols = sample50.columns.tolist()
drop_idx = [0,1,5,6,11]
drop_cols = [c for i,c in enumerate(cols) if i in drop_idx]
sample50_clean = sample50.drop(columns=drop_cols, errors='ignore')
os.makedirs('data/processed', exist_ok=True)
path_xlsx = 'data/processed/sample50.xlsx'
sample50_clean.to_excel(path_xlsx, index=False)
pd.read_excel(path_xlsx).head()

## 1.2 Visualizaciones
a) Mostrar todas las filas (y luego restaurar).  
b) Mostrar todas las columnas (y luego restaurar).  
c) Gráfica de líneas: fallecidos, confirmados, recuperados, activos por país con fallecidos > 2500.  
d) Barras: fallecidos por estados de EE.UU.  
e) Pie: fallecidos de COL, CHL, PER, ARG, MEX.  
f) Histograma: fallecidos por país.  
g) Boxplot: Confirmed, Deaths, Recovered, Active.


In [ ]:
# a) todas las filas
_old_max_rows = pd.get_option('display.max_rows')
pd.set_option('display.max_rows', None)
df
pd.set_option('display.max_rows', _old_max_rows)

# b) todas las columnas
_old_max_cols = pd.get_option('display.max_columns')
pd.set_option('display.max_columns', None)
df.head(3)
pd.set_option('display.max_columns', _old_max_cols)

In [ ]:
# c) líneas por país con muertes > 2500 (usando valores totales por país)
thr = 2500
big = by_country[by_country['Deaths']>thr]
for col in ['Deaths','Confirmed','Recovered','Active']:
    plt.figure()
    plt.plot(big['Country_Region'], big[col])
    plt.title(f'{col} por país (muertes>{thr})')
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

In [ ]:
# d) Barras: fallecidos por estados de EE.UU.
us = by_cp[by_cp['Country_Region']=='US'].sort_values('Deaths', ascending=False)
plt.figure(); plt.bar(us['Province_State'].astype(str).head(30), us['Deaths'].head(30));
plt.title('Fallecidos por estado de EE.UU. (Top 30)'); plt.xticks(rotation=90); plt.tight_layout(); plt.show()

In [ ]:
# e) Pie países específicos
paises = ['Colombia','Chile','Peru','Argentina','Mexico']
sel = by_country[by_country['Country_Region'].isin(paises)][['Country_Region','Deaths']]
plt.figure(); plt.pie(sel['Deaths'], labels=sel['Country_Region'], autopct='%1.1f%%'); plt.title('Fallecidos seleccionados'); plt.show()

In [ ]:
# f) Histograma de fallecidos por país
plt.figure(); plt.hist(by_country['Deaths'], bins=30); plt.title('Histograma de fallecidos por país'); plt.show()

# g) Boxplots
plt.figure(); df[['Confirmed','Deaths','Recovered','Active']].plot(kind='box'); plt.title('Boxplots'); plt.show()

## 2. Estadística avanzada
- Métricas por país (CFR, tasas 100k)
- IC Wilson (95%) para CFR
- Test de dos proporciones entre dos países
- Outliers por Z-score e IQR
- Gráfico de control (3σ) de muertes diarias


In [ ]:
def country_metrics(dfx: pd.DataFrame) -> pd.DataFrame:
    g = dfx.groupby('Country_Region', dropna=False).agg(
        Confirmed=('Confirmed','sum'), Deaths=('Deaths','sum'),
        Recovered=('Recovered','sum'), Active=('Active','sum'),
        Incident_Rate=('Incident_Rate','mean'), CFR_mean=('Case_Fatality_Ratio','mean')
    ).reset_index()
    g['CFR'] = np.where(g['Confirmed']>0, g['Deaths']/g['Confirmed'], np.nan)
    if 'Population' in dfx.columns and dfx['Population'].notna().any():
        pop = dfx.groupby('Country_Region')['Population'].sum().rename('Population').reset_index()
        g = g.merge(pop, on='Country_Region', how='left')
        g['Confirmed_100k'] = g['Confirmed']/g['Population']*1e5
        g['Deaths_100k'] = g['Deaths']/g['Population']*1e5
    else:
        g['Confirmed_100k'] = g['Incident_Rate']
        g['Deaths_100k'] = g['CFR']*g['Confirmed_100k']
    return g

cm = country_metrics(df)
cm.head()

In [ ]:
# IC Wilson (95%) para CFR
def wilson_ci(D, C, z=1.96):
    if C==0: return np.nan, np.nan, np.nan
    p = D/C
    denom = 1 + z**2/C
    center = p + z**2/(2*C)
    rad = z * math.sqrt(p*(1-p)/C + (z**2)/(4*C**2))
    lo = (center - rad)/denom
    hi = (center + rad)/denom
    return p, lo, hi

ci = cm.apply(lambda r: wilson_ci(r['Deaths'], r['Confirmed']), axis=1, result_type='expand')
cm[['CFR','CFR_L','CFR_U']] = ci
cm[['Country_Region','CFR','CFR_L','CFR_U']].head()

In [ ]:
# Test de dos proporciones (CFR) entre dos países
def prop_test_2countries(cm, A='Peru', B='Chile'):
    a = cm.loc[cm['Country_Region']==A, ['Confirmed','Deaths']].iloc[0]
    b = cm.loc[cm['Country_Region']==B, ['Confirmed','Deaths']].iloc[0]
    C1,D1 = int(a['Confirmed']), int(a['Deaths'])
    C2,D2 = int(b['Confirmed']), int(b['Deaths'])
    if C1==0 or C2==0:
        return np.nan, np.nan
    phat = (D1 + D2)/(C1 + C2)
    z = (D1/C1 - D2/C2)/np.sqrt(phat*(1-phat)*(1/C1 + 1/C2))
    pval = 2*(1 - stats.norm.cdf(abs(z)))
    return z, pval

prop_test_2countries(cm, 'Peru', 'Chile')

In [ ]:
# Outliers: Z-score e IQR
def outliers_z(series):
    z = (series - series.mean())/series.std(ddof=0)
    return series.index[(z.abs()>3)].tolist()

def outliers_iqr(series):
    q1, q3 = series.quantile([0.25, 0.75])
    iqr = q3 - q1
    lo = q1 - 1.5*iqr
    hi = q3 + 1.5*iqr
    return series.index[(series<lo)|(series>hi)].tolist()

for col in ['Confirmed_100k','Deaths_100k','CFR']:
    z_out = outliers_z(cm[col].dropna())
    iqr_out = outliers_iqr(cm[col].dropna())
    print(col, 'Z-outliers:', len(z_out), 'IQR-outliers:', len(iqr_out))

## 3. Series de tiempo y forecast
Se arma una ventana histórica de daily reports para construir series por país y pronosticar **muertes** a 14 días (SARIMA).

In [ ]:
def load_daily(tag: str) -> pd.DataFrame:
    url = f"https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{tag}.csv"
    x = pd.read_csv(url)
    x = x.rename(columns={'Province/State':'Province_State','Country/Region':'Country_Region','Last Update':'Last_Update','Latitude':'Lat','Longitude':'Long_'})
    return x

def load_range(end_tag: str, days_back: int = 90) -> pd.DataFrame:
    end = datetime.strptime(end_tag, '%m-%d-%Y')
    frames = []
    for i in range(days_back+1):
        d = end - timedelta(days=i)
        tag = d.strftime('%m-%d-%Y')
        try:
            df = load_daily(tag)
            df['date'] = d.date()
            frames.append(df)
        except Exception:
            pass
    return pd.concat(frames, ignore_index=True) if frames else pd.DataFrame()

hist = load_range(DATE_STR, 120)
hist.head()

In [ ]:
# Serie de muertes diarias por país (agregado)
def country_series(hist: pd.DataFrame, country: str) -> pd.Series:
    ts = hist.groupby(['date','Country_Region'])['Deaths'].sum().reset_index()
    y = ts.loc[ts['Country_Region']==country, ['date','Deaths']].set_index('date').sort_index()
    y = y.asfreq('D').fillna(0)['Deaths']
    return y

y = country_series(hist, 'Peru')
y_7 = y.rolling(7).mean()
ax = y.plot(title='Peru - muertes diarias'); y_7.plot(ax=ax)
plt.legend(['Deaths','7d MA']); plt.show()

In [ ]:
# SARIMA auto_arima + backtesting de 14 días
if len(y) > 30:
    model = pm.auto_arima(y, seasonal=False, stepwise=True, suppress_warnings=True)
    fc, conf = model.predict(n_periods=14, return_conf_int=True)
    idx_fc = pd.date_range(y.index.max() + pd.Timedelta(days=1), periods=14, freq='D')
    plt.figure(); y.plot(); plt.plot(idx_fc, fc)
    plt.fill_between(idx_fc, conf[:,0], conf[:,1], alpha=0.2)
    plt.title('Pronóstico 14 días'); plt.show()

    # backtesting
    split = -14
    train, test = y[:split], y[split:]
    m = pm.auto_arima(train, seasonal=False, stepwise=True, suppress_warnings=True)
    pred = pd.Series(m.predict(n_periods=len(test)), index=test.index)
    mae = (test - pred).abs().mean()
    mape = ((test - pred).abs()/(test.replace(0, np.nan))).mean()*100
    print('MAE:', round(mae,2), 'MAPE:', round(mape,2),'%')
else:
    print('Serie insuficiente para modelado.')

## 4. Clustering y PCA
Variables: Confirmed_100k, Deaths_100k, CFR. K-means con k=3 por defecto; PCA a 2 componentes.

In [ ]:
X = cm[['Confirmed_100k','Deaths_100k','CFR']].replace([np.inf,-np.inf], np.nan).dropna()
idx = X.index
Xs = StandardScaler().fit_transform(X)
km = KMeans(n_clusters=3, n_init='auto', random_state=42)
lab = km.fit_predict(Xs)
pca = PCA(n_components=2, random_state=42)
PC = pca.fit_transform(Xs)
scatter = pd.DataFrame({'PC1':PC[:,0],'PC2':PC[:,1],'cluster':lab,'Country':cm.loc[idx,'Country_Region'].values})
scatter.head()

## 5. Gráfico de control (3σ)
Aplicado a muertes diarias por país (ej. Peru).

In [ ]:
def control_chart(series: pd.Series, window=14):
    roll = series.rolling(window)
    mu, sigma = roll.mean(), roll.std().replace(0, np.nan)
    cl = mu; ucl = mu + 3*sigma; lcl = (mu - 3*sigma).clip(lower=0)
    ax = series.plot(label='Deaths'); cl.plot(ax=ax, label='CL'); ucl.plot(ax=ax, label='UCL'); lcl.plot(ax=ax, label='LCL')
    plt.legend(); plt.title('Control chart (3σ)'); plt.show()

control_chart(y)